In [ ]:
import pandas as pd
import re

# Load COVID (FIGHTING AND INFODEMIC) DFs
covid_train_data = pd.read_csv('Cross_Validation/Binary/train.csv')
covid_test_data = pd.read_csv('Cross_Validation/Binary/test.csv')
covid_val_data = pd.read_csv('Cross_Validation/Binary/val.csv')

# Combine into a Single Dataset
COVID_combined_data = pd.concat([covid_train_data, covid_test_data, covid_val_data], ignore_index=True)
COVID_combined_data

In [ ]:
COVID_combined_data['target'] = COVID_combined_data['label'].apply(lambda x: 0 if x == 'fake' else 1)

COVID_combined_data = COVID_combined_data.drop(columns=['label', 'id'])
COVID_combined_data

In [ ]:
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

def preprocess_text(text, number_indicator="NUM", url_indicator="URL", percent_indicator="PERCENT", money_indicator="MONEY"):
    
    punctuation = r"[\]^_`{|}&<>*#-:"

    # Function to expand contractions
    def replace_contractions(text):
        contractions_pattern = re.compile(r'\b(' + '|'.join(re.escape(key) for key in contraction_dict.keys()) + r')\b', re.IGNORECASE)

        def expand_match(contraction):
            match = contraction.group(0)
            first_char = match[0]
            expanded_contraction = contraction_dict[match.lower() if match.lower() in contraction_dict else match]
            return first_char + expanded_contraction[1:] if first_char.isupper() else expanded_contraction

        return contractions_pattern.sub(expand_match, text)

    # Expand contractions
    text = replace_contractions(text)

    # Remove punctuation
    for p in punctuation:
        text = text.replace(p, "")

    # Replace URLs with URL
    url_pattern = r'https?//(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    text = re.sub(url_pattern, url_indicator, text)

    # Label MONEY
    text = re.sub(r'\$\b(?!0+(?:\.0+)?$)\d+(\.\d{1,2})?\b', lambda m: money_indicator, text)

    # Replace numbers, except years, with NUM 
    text = re.sub(r'\b\d{1,3}\b', number_indicator, text)
    text = re.sub(r'\b(?!1|2)\d{4,}\b', number_indicator, text)
    
    # Label PERCENT
    text = text.replace(number_indicator + "%", percent_indicator)
    
    # Replace newline with a space and remove mentions
    text = text.replace('\n', ' ').replace('\r', ' ')
    text = re.sub(r'@\w+', '', text)

    return text

In [ ]:
COVID_combined_data['tweet'] = COVID_combined_data['tweet'].apply(lambda x: pd.Series(preprocess_text(x)))
COVID_combined_data

In [ ]:
COVID_combined_data[['tweet', 'target']].to_json('Cross_Validation/COVID_processed.json', orient='records')

### Multiclass (3)

In [ ]:
covid_true_data = pd.read_csv('Cross_Validation/Multiclass_3/English_True.csv', encoding='ISO-8859-1')
covid_false_data = pd.read_csv('Cross_Validation/Multiclass_3/English_False.csv', encoding='ISO-8859-1')
covid_uncertain_data = pd.read_csv('Cross_Validation/Multiclass_3/English_Uncertain.csv', encoding='ISO-8859-1')

In [ ]:
covid_true_data

In [ ]:
covid_true_data = covid_true_data.iloc[1:]
covid_true_data = covid_true_data.drop(covid_true_data.columns[0], axis=1)

covid_false_data = covid_false_data.iloc[1:]
covid_false_data = covid_false_data.drop(covid_false_data.columns[0], axis=1)

covid_uncertain_data = covid_uncertain_data.iloc[1:]
covid_uncertain_data = covid_uncertain_data.drop(covid_uncertain_data.columns[0], axis=1)

covid_true_data

In [ ]:
covid_true_data.rename(columns={'Unnamed: 1': 'tweet'}, inplace=True)
covid_true_data['target'] = 'true'

covid_false_data.rename(columns={'Unnamed: 1': 'tweet'}, inplace=True)
covid_false_data['target'] = 'false'

covid_uncertain_data.rename(columns={'Unnamed: 1': 'tweet'}, inplace=True)
covid_uncertain_data['target'] = 'no_majority'

covid_true_data

In [ ]:
COVID_multiclass_data = pd.concat([covid_true_data, covid_false_data, covid_uncertain_data], ignore_index=True)
COVID_multiclass_data

In [ ]:
COVID_multiclass_data['tweet'] = COVID_multiclass_data['tweet'].apply(lambda x: pd.Series(preprocess_text(x)))
COVID_multiclass_data

In [ ]:
COVID_multiclass_data = COVID_multiclass_data.sample(frac=1).reset_index(drop=True)

In [ ]:
COVID_multiclass_data

In [ ]:
COVID_multiclass_data[['tweet', 'target']].to_json('Cross_Validation/COVID_processed_multiclass_3.json', orient='records')